In [1]:
import time
import os
import numba

from coffea.nanoevents import BaseSchema

import awkward as ak
import numpy as np
from coffea import processor, hist

from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import mplhep
import matplotlib.pyplot as plt
plt.style.use(mplhep.style.CMS)

In [10]:
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx']

dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl',
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

d0_cols = ['D0_pt', 'D0_eta', 'D0_phi', 'D0_rap', 'D0_mass12', 'D0_mass21', 'D0_vtxIdx', 'D0_chi2', 'D0_dl', 'D0_dlErr', 'D0_dlSig',
           'D0_cosphi', 'D0_x', 'D0_y', 'D0_z', 'D0_hasMuon',
           'D0_t1pt', 'D0_t1eta', 'D0_t1phi', 'D0_t1chindof', 'D0_t1nValid', 'D0_t1nPix', 'D0_t1dxy', 'D0_t1dz', 'D0_t1chg', 
           'D0_t2pt', 'D0_t2eta', 'D0_t2phi', 'D0_t2chindof', 'D0_t2nValid', 'D0_t2nPix', 'D0_t2dxy', 'D0_t2dz', 'D0_t2chg',]

dstar_cols = ['Dstar_pt', 'Dstar_eta', 'Dstar_phi', 'Dstar_rap', 'Dstar_deltam', 'Dstar_deltamr', 'Dstar_vtxIdx', 'Dstar_hasMuon',
              'Dstar_D0pt', 'Dstar_D0eta', 'Dstar_D0phi', 'Dstar_D0mass', 'Dstar_D0chi2', 'Dstar_D0dl', 'Dstar_D0dlErr',
              'Dstar_D0dlSig', 'Dstar_D0cosphi', 'Dstar_D0x', 'Dstar_D0y', 'Dstar_D0z',
              'Dstar_Kpt', 'Dstar_Keta', 'Dstar_Kphi', 'Dstar_KvtxIdx', 'Dstar_Kchindof', 'Dstar_KnValid', 'Dstar_KnPix', 'Dstar_Kdxy',
              'Dstar_Kdz', 'Dstar_Kchg',
              'Dstar_pipt', 'Dstar_pieta', 'Dstar_piphi', 'Dstar_pivtxIdx', 'Dstar_pichindof', 'Dstar_pinValid', 'Dstar_pinPix',
              'Dstar_pidxy', 'Dstar_pidz', 'Dstar_pichg',
              'Dstar_pispt', 'Dstar_piseta', 'Dstar_pisphi', 'Dstar_pisvtxIdx', 'Dstar_pischindof', 'Dstar_pisnValid', 'Dstar_pisnPix',
              'Dstar_pisdxy', 'Dstar_pisdz', 'Dstar_simIdx', 'Dstar_D0simIdx']

gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 'GenPart_status',
                'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 'GenPart_nstchgdaug', 'GenPart_vx', 
                'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 'GenPart_recIdx']

D0_PDG_MASS = 1.864

def get_vars_dict(events, col_list):
    dict = {}
    col = ''
    for c in col_list:
        if c.startswith('Muon'):
            col = c[5:]
        elif c.startswith('Dimu'):
            col = c[4:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('D0'):
            col = c[2:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('Dstar'):
            col = c[5:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('PVtx'):
            col = c[5:]
        elif c.startswith("GenPart"):
            col = c[8:]
        else:
            Exception('Not good!')

        if col == 'x' or col == 'y' or col == 'z':
            col = 'vtx_' + col

        if len(events[c]) == 0:
            dict[col] = np.array([])
        else:
            dict[col] = events[c]
    return dict

In [3]:
import re

paths = ['/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/211111_200135/0000',
         '/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/211111_200135/0001']
files = []
for path in paths:
    with os.scandir(path) as it:
        for file in it:
            if file.name.endswith('.root') and (file.stat().st_size != 0):
                files.append(file.path)

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
            
files.sort(key=natural_keys)

FileNotFoundError: [Errno 2] No such file or directory: '/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/211111_200135/0000'

In [11]:
class GenTestProcessor(processor.ProcessorABC):
    def __init__(self):
        self._accumulator = processor.dict_accumulator({
            'details': processor.defaultdict_accumulator(int),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        output = self.accumulator.identity()
        
        # test if there is any events in the file
        if len(events) == 0:
            return output

        output['details']['n_events'] += len(events)
        
        # Collection extraction
        GenPart = ak.zip({**get_vars_dict(events, gen_part_cols)}, with_name="PtEtaPhiMCandidate")
        
        GenPart_ups = GenPart[((GenPart.pdgId == 553) | (GenPart.pdgId == 100553) | (GenPart.pdgId == 200553))]
        ups_2dau_id = GenPart_ups[GenPart_ups.numberOfDaughters == 2].Id
        GenPart_muon = GenPart[np.absolute(GenPart.pdgId) == 13]
        GenPart_dstar = GenPart[np.absolute(GenPart.pdgId) == 413]
        GenPart_d0 = GenPart[np.absolute(GenPart.pdgId) == 421]

        print(f'Starting Event: {events.event[0]}')
        for i0 in range(len(GenPart_ups)):
            print(f"Event {i0}:")
            for i1 in range(len(GenPart_ups[i0])):
                print(f"Y: {GenPart_ups[i0][i1].pdgId}, ndaughters: {GenPart_ups[i0][i1].numberOfDaughters}, ", end='')
                print(f"Id: {GenPart_ups[i0][i1].Id}, motherId: {GenPart_ups[i0][i1].genPartIdxMother}, status: {GenPart_ups[i0][i1].status}")
                if GenPart_ups[i0][i1].numberOfDaughters == 2:
                    print('2 Daughters!!')
                    output['details']['n_upsilon'] += 1
            for i1 in range(len(GenPart_muon[i0])):
                print(f"Muon: {GenPart_muon[i0][i1].pdgId}, ndaughters: {GenPart_muon[i0][i1].numberOfDaughters}, ", end='')
                print(f"Id: {GenPart_muon[i0][i1].Id}, motherId: {GenPart_muon[i0][i1].genPartIdxMother}, status: {GenPart_muon[i0][i1].status}")
                if GenPart_muon[i0][i1].genPartIdxMother in ups_2dau_id[i0]:
                    print('From Y!!')
                    output['details']['n_muon'] += 1
                else:
                    print('Not from Y :(')
            for i1 in range(len(GenPart_dstar[i0])):
                print(f"D*: {GenPart_dstar[i0][i1].pdgId}, ndaughters: {GenPart_dstar[i0][i1].numberOfDaughters}, ", end='')
                print(f"Id: {GenPart_dstar[i0][i1].Id}, motherId: {GenPart_dstar[i0][i1].genPartIdxMother}, status: {GenPart_dstar[i0][i1].status}")
                output['details']['n_dstar'] += 1
            for i1 in range(len(GenPart_d0[i0])):
                print(f"D0: {GenPart_d0[i0][i1].pdgId}, ndaughters: {GenPart_d0[i0][i1].numberOfDaughters}, ", end='')
                print(f"Id: {GenPart_d0[i0][i1].Id}, motherId: {GenPart_d0[i0][i1].genPartIdxMother}, status: {GenPart_d0[i0][i1].status}")
                if GenPart_d0[i0][i1].genPartIdxMother in GenPart_dstar[i0].Id:
                    print("From D*!!")
                    output['details']['n_d0'] += 1
                else:
                    print('Not from D* :(')
            print('')
        print(f'Ending Event: {events.event[-1]}')

        return output

    def postprocess(self, accumulator):
        return accumulator

In [14]:
data = {"test": ["/afs/cern.ch/work/k/kmotaama/public/analysis/OniaOpenCharmRun2UL/OniaOpenCharmRun2ULMC/CMSSW_10_6_12/src/genTuple/genTuple/test.root"]}
# data = {"test": files[:200]}

tstart = time.time()

output = processor.run_uproot_job(data,
                                  treename='genTuple/Events',
                                  processor_instance=GenTestProcessor(),
                                  executor=processor.iterative_executor,
                                  executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                  chunksize=360000,
                                  )

print(f"Process finished in: {time.time() - tstart:.2f} s")
#print(output['cutflow'])

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 3163.13file/s]


TypeError: 'NoneType' object is not subscriptable

In [7]:
print(output)

NameError: name 'output' is not defined

In [18]:
!ls /afs/cern.ch/work/k/kmotaama/public/analysis/OniaOpenCharmRun2UL/OniaOpenCharmRun2ULMC/CMSSW_10_6_12/src/genTuple/

ls: /afs/cern.ch/work/k/kmotaama/public/analysis/OniaOpenCharmRun2UL/OniaOpenCharmRun2ULMC/CMSSW_10_6_12/src/genTuple/: No such file or directory


In [2]:
corrections = ak.Array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
Upsilons = ak.Array([[1,2],[3],[4,5,6],[],[7],[8,9]])

In [9]:
nUpsilon = ak.num(Upsilons)
correcto = np.repeat(corrections, nUpsilon)

for i, c in enumerate(correcto):
    print(c, ak.flatten(Upsilons)[i])

0.1 1
0.1 2
0.2 3
0.3 4
0.3 5
0.3 6
0.5 7
0.6 8
0.6 9
